# Lab 4.6.5: Option D - Custom Training Pipeline

**Module:** 4.6 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build **infrastructure for continuous model improvement**:

| Component | Description | DGX Spark Advantage |
|-----------|-------------|---------------------|
| **Data Pipeline** | Collection & curation | High-speed preprocessing |
| **Multi-stage Training** | SFT → DPO → RLHF | Full fine-tuning of 16B models |
| **Automated Eval** | Benchmark suite | Run evals without moving models |
| **Model Registry** | Versioning & comparison | Store multiple model versions |
| **A/B Testing** | Compare model variants | Side-by-side inference |
| **Deployment** | Automated model updates | Fast model swapping |

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Build a data collection and curation pipeline
- [ ] Implement multiple fine-tuning approaches (SFT, DPO)
- [ ] Create automated evaluation benchmarks
- [ ] Build model versioning and comparison tools
- [ ] Implement A/B testing framework
- [ ] Create automated deployment pipelines

---

## 🌍 Real-World Context

ML training pipelines are the backbone of AI companies:

| Company | Pipeline Scale | Key Features |
|---------|---------------|---------------|
| **OpenAI** | Thousands of experiments | RLHF, Constitutional AI |
| **Anthropic** | Continuous training | CAI, safety training |
| **Google** | Distributed training | Gemini, multimodal |
| **Meta** | Open models | Llama training infrastructure |
| **Hugging Face** | Community | AutoTrain, evaluation hub |

A good training pipeline can reduce iteration time from weeks to hours.

---

## 🧒 ELI5: What is a Training Pipeline?

> **Imagine you're running a cooking school** that needs to get better over time:
>
> 1. **Collect recipes** (data collection) - Gather the best recipes from everywhere
> 2. **Test recipes** (data curation) - Remove bad ones, keep good ones
> 3. **Train students** (fine-tuning) - Teach them with the good recipes
> 4. **Grade students** (evaluation) - See who learned best
> 5. **Pick the best** (model selection) - Graduate the top performers
> 6. **Open restaurants** (deployment) - Put them to work!
> 7. **Get feedback** (monitoring) - Learn what to improve
> 8. **Repeat!** - Keep getting better
>
> A **training pipeline** automates this whole process for AI models:
> - New data comes in automatically
> - Models are trained and tested automatically
> - The best models are deployed automatically
> - Everything is tracked and versioned
>
> The result? Your AI gets better continuously without manual intervention.

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      Custom Training Pipeline                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                     DATA PIPELINE                                 │   │
│  │  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐    │   │
│  │  │Collect │─▶│ Clean  │─▶│ Format │─▶│ Split  │─▶│ Store  │    │   │
│  │  └────────┘  └────────┘  └────────┘  └────────┘  └────────┘    │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                               │                                          │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                    TRAINING STAGES                                │   │
│  │  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐                 │   │
│  │  │  SFT   │─▶│  DPO   │─▶│ Safety │─▶│ Merge  │                 │   │
│  │  │Training│  │Training│  │Training│  │Adapters│                 │   │
│  │  └────────┘  └────────┘  └────────┘  └────────┘                 │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                               │                                          │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                    EVALUATION & REGISTRY                          │   │
│  │  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐                 │   │
│  │  │Bench-  │─▶│Compare │─▶│Register│─▶│Version │                 │   │
│  │  │marks   │  │Baseline│  │ Model  │  │Control │                 │   │
│  │  └────────┘  └────────┘  └────────┘  └────────┘                 │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                               │                                          │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │                    DEPLOYMENT & TESTING                           │   │
│  │  ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐                 │   │
│  │  │A/B Test│─▶│ Deploy │─▶│Monitor │─▶│Feedback│                 │   │
│  │  │ Setup  │  │ Winner │  │ Metrics│  │  Loop  │                 │   │
│  │  └────────┘  └────────┘  └────────┘  └────────┘                 │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Environment Setup
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
import json
import hashlib

import torch

print("🔧 OPTION D: CUSTOM TRAINING PIPELINE")
print("="*70)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Memory Planning for Training Pipeline

def plan_option_d_memory():
    """
    Plan memory for training pipeline.
    
    Key insight: We don't need all components loaded simultaneously.
    Pipeline stages run sequentially.
    """
    
    print("\n💾 DGX SPARK MEMORY PLAN - OPTION D")
    print("="*70)
    print("Strategy: Sequential stages, not concurrent")
    
    stages = [
        {
            "name": "Stage 1: Data Processing",
            "components": [
                ("Data loading buffers", 10.0),
                ("Text processing", 4.0),
                ("Tokenization cache", 6.0),
            ],
        },
        {
            "name": "Stage 2: SFT Training (16B Model)",
            "components": [
                ("Base model (16B INT4)", 10.0),
                ("LoRA adapters", 2.0),
                ("Optimizer states", 8.0),
                ("Activations (grad ckpt)", 12.0),
                ("Data loader", 4.0),
            ],
        },
        {
            "name": "Stage 3: DPO Training",
            "components": [
                ("Policy model (16B INT4)", 10.0),
                ("Reference model (16B INT4)", 10.0),
                ("LoRA adapters", 2.0),
                ("Optimizer states", 8.0),
                ("Preference pairs", 4.0),
            ],
        },
        {
            "name": "Stage 4: Evaluation",
            "components": [
                ("Model under test (16B)", 10.0),
                ("Eval datasets", 2.0),
                ("Metrics computation", 2.0),
            ],
        },
        {
            "name": "Stage 5: A/B Testing (Inference)",
            "components": [
                ("Model A (16B INT4)", 10.0),
                ("Model B (16B INT4)", 10.0),
                ("Routing logic", 1.0),
            ],
        },
    ]
    
    max_stage_memory = 0
    
    for stage in stages:
        stage_total = sum(c[1] for c in stage["components"])
        max_stage_memory = max(max_stage_memory, stage_total)
        
        print(f"\n📊 {stage['name']}")
        print("-"*50)
        for name, mem in stage["components"]:
            print(f"   {name:<30} {mem:>6.1f} GB")
        print(f"   {'STAGE TOTAL':<30} {stage_total:>6.1f} GB")
    
    print("\n" + "="*70)
    print(f"Peak memory (Stage 3 - DPO): {max_stage_memory:.1f} GB")
    print(f"DGX Spark available: 128 GB")
    print(f"Headroom: {128 - max_stage_memory:.1f} GB")
    print("\n✅ All stages fit! Can even use larger models (up to ~40B for SFT)")
    
    return max_stage_memory

plan_option_d_memory()

---

## Part 2: Data Pipeline

In [ ]:
# Data Pipeline Components

@dataclass
class DataSample:
    """A single data sample."""
    id: str
    content: Dict[str, Any]  # instruction, input, output, etc.
    source: str
    quality_score: float = 0.0
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class Dataset:
    """A dataset with samples."""
    name: str
    samples: List[DataSample]
    created_at: str
    version: str
    split: str = "train"  # train, val, test

class DataPipeline:
    """
    Pipeline for collecting, cleaning, and preparing training data.
    
    Stages:
    1. Collection - Gather raw data from sources
    2. Cleaning - Remove duplicates, filter quality
    3. Formatting - Convert to training format
    4. Splitting - Create train/val/test splits
    5. Versioning - Track dataset versions
    """
    
    def __init__(self, data_dir: str = "./data"):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(parents=True, exist_ok=True)
        self.sample_id_counter = 0
    
    def collect_from_jsonl(self, file_path: str, source: str = "jsonl") -> List[DataSample]:
        """
        Collect samples from a JSONL file.
        """
        samples = []
        with open(file_path, 'r') as f:
            for line in f:
                if line.strip():
                    self.sample_id_counter += 1
                    data = json.loads(line)
                    samples.append(DataSample(
                        id=f"sample_{self.sample_id_counter}",
                        content=data,
                        source=source,
                    ))
        print(f"✅ Collected {len(samples)} samples from {file_path}")
        return samples
    
    def clean_samples(
        self,
        samples: List[DataSample],
        min_length: int = 10,
        max_length: int = 10000,
        remove_duplicates: bool = True,
    ) -> List[DataSample]:
        """
        Clean and filter samples.
        """
        cleaned = []
        seen_hashes = set()
        
        for sample in samples:
            # Get text content
            text = json.dumps(sample.content)
            
            # Length filter
            if len(text) < min_length or len(text) > max_length:
                continue
            
            # Deduplication
            if remove_duplicates:
                text_hash = hashlib.md5(text.encode()).hexdigest()
                if text_hash in seen_hashes:
                    continue
                seen_hashes.add(text_hash)
            
            cleaned.append(sample)
        
        print(f"✅ Cleaned: {len(samples)} → {len(cleaned)} samples")
        return cleaned
    
    def format_for_sft(
        self,
        samples: List[DataSample],
        template: str = "alpaca",
    ) -> List[Dict]:
        """
        Format samples for SFT training.
        
        Templates:
        - alpaca: instruction, input, output
        - chatml: messages list
        - raw: plain text
        """
        formatted = []
        
        for sample in samples:
            content = sample.content
            
            if template == "alpaca":
                text = f"""### Instruction:
{content.get('instruction', '')}

### Input:
{content.get('input', '')}

### Response:
{content.get('output', '')}"""
            elif template == "chatml":
                messages = content.get('messages', [])
                text = "\n".join([
                    f"<|im_start|>{m['role']}\n{m['content']}<|im_end|>"
                    for m in messages
                ])
            else:
                text = content.get('text', str(content))
            
            formatted.append({"text": text, "id": sample.id})
        
        print(f"✅ Formatted {len(formatted)} samples for SFT ({template} template)")
        return formatted
    
    def format_for_dpo(
        self,
        samples: List[DataSample],
    ) -> List[Dict]:
        """
        Format samples for DPO training.
        
        Expects samples with 'chosen' and 'rejected' responses.
        """
        formatted = []
        
        for sample in samples:
            content = sample.content
            if 'chosen' in content and 'rejected' in content:
                formatted.append({
                    "prompt": content.get('prompt', content.get('instruction', '')),
                    "chosen": content['chosen'],
                    "rejected": content['rejected'],
                    "id": sample.id,
                })
        
        print(f"✅ Formatted {len(formatted)} samples for DPO")
        return formatted
    
    def create_splits(
        self,
        samples: List[DataSample],
        train_ratio: float = 0.9,
        val_ratio: float = 0.05,
        seed: int = 42,
    ) -> Dict[str, List[DataSample]]:
        """
        Create train/val/test splits.
        """
        import random
        random.seed(seed)
        
        shuffled = samples.copy()
        random.shuffle(shuffled)
        
        n = len(shuffled)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))
        
        splits = {
            "train": shuffled[:train_end],
            "val": shuffled[train_end:val_end],
            "test": shuffled[val_end:],
        }
        
        print(f"✅ Created splits: train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")
        return splits
    
    def save_dataset(
        self,
        samples: List[Dict],
        name: str,
        version: str = "v1.0",
    ) -> str:
        """
        Save dataset to disk.
        """
        output_path = self.data_dir / f"{name}_{version}.jsonl"
        
        with open(output_path, 'w') as f:
            for sample in samples:
                f.write(json.dumps(sample) + '\n')
        
        print(f"✅ Saved dataset to {output_path}")
        return str(output_path)

print("✅ DataPipeline class defined")

---

## Part 3: Training Stages

In [ ]:
# Training Stage Configuration

class TrainingStage(Enum):
    SFT = "sft"          # Supervised Fine-Tuning
    DPO = "dpo"          # Direct Preference Optimization
    ORPO = "orpo"        # Odds Ratio Preference Optimization
    KTO = "kto"          # Kahneman-Tversky Optimization

@dataclass
class TrainingConfig:
    """Configuration for a training run."""
    stage: TrainingStage
    base_model: str
    dataset_path: str
    output_dir: str
    
    # LoRA config
    lora_r: int = 64
    lora_alpha: int = 128
    lora_dropout: float = 0.05
    
    # Training config
    num_epochs: int = 3
    batch_size: int = 2
    gradient_accumulation: int = 8
    learning_rate: float = 2e-4
    max_seq_length: int = 2048
    
    # DPO specific
    dpo_beta: float = 0.1

@dataclass
class TrainingResult:
    """Result of a training run."""
    run_id: str
    stage: TrainingStage
    model_path: str
    metrics: Dict[str, float]
    duration_seconds: float
    config: TrainingConfig
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

class TrainingOrchestrator:
    """
    Orchestrates multi-stage training runs.
    
    Supports:
    - SFT (Supervised Fine-Tuning)
    - DPO (Direct Preference Optimization)
    - Sequential stage execution
    - Checkpoint management
    """
    
    def __init__(self, base_dir: str = "./training_runs"):
        self.base_dir = Path(base_dir)
        self.base_dir.mkdir(parents=True, exist_ok=True)
        self.runs: List[TrainingResult] = []
        self._run_id_counter = 0
    
    def run_sft(
        self,
        config: TrainingConfig,
        resume_from: str = None,
    ) -> TrainingResult:
        """
        Run SFT training.
        
        In production, this would use TRL's SFTTrainer.
        """
        import time
        
        self._run_id_counter += 1
        run_id = f"sft_run_{self._run_id_counter:04d}"
        
        print(f"\n🚀 Starting SFT Training: {run_id}")
        print(f"   Model: {config.base_model}")
        print(f"   Dataset: {config.dataset_path}")
        print(f"   Epochs: {config.num_epochs}")
        
        start = time.time()
        
        # Simulated training (in production, use TRL)
        training_code = f'''
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

# Load model
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained("{config.base_model}", quantization_config=bnb_config)

# LoRA
lora_config = LoraConfig(r={config.lora_r}, lora_alpha={config.lora_alpha})
model = get_peft_model(model, lora_config)

# Train
trainer = SFTTrainer(model=model, train_dataset=dataset, ...)
trainer.train()
trainer.save_model("{config.output_dir}")
'''
        
        # Placeholder metrics
        metrics = {
            "train_loss": 0.45,
            "eval_loss": 0.52,
            "perplexity": 3.2,
        }
        
        duration = time.time() - start
        
        result = TrainingResult(
            run_id=run_id,
            stage=TrainingStage.SFT,
            model_path=config.output_dir,
            metrics=metrics,
            duration_seconds=duration,
            config=config,
        )
        
        self.runs.append(result)
        print(f"✅ SFT Complete. Loss: {metrics['train_loss']:.4f}")
        
        return result
    
    def run_dpo(
        self,
        config: TrainingConfig,
        sft_model_path: str = None,
    ) -> TrainingResult:
        """
        Run DPO training.
        
        Requires preference data with (prompt, chosen, rejected).
        """
        import time
        
        self._run_id_counter += 1
        run_id = f"dpo_run_{self._run_id_counter:04d}"
        
        base = sft_model_path or config.base_model
        
        print(f"\n🚀 Starting DPO Training: {run_id}")
        print(f"   Policy Model: {base}")
        print(f"   Beta: {config.dpo_beta}")
        
        start = time.time()
        
        # Placeholder metrics
        metrics = {
            "train_loss": 0.35,
            "chosen_reward": 0.8,
            "rejected_reward": -0.5,
            "reward_margin": 1.3,
        }
        
        duration = time.time() - start
        
        result = TrainingResult(
            run_id=run_id,
            stage=TrainingStage.DPO,
            model_path=config.output_dir,
            metrics=metrics,
            duration_seconds=duration,
            config=config,
        )
        
        self.runs.append(result)
        print(f"✅ DPO Complete. Reward margin: {metrics['reward_margin']:.4f}")
        
        return result

print("✅ TrainingOrchestrator class defined")

---

## Part 4: Model Registry

In [ ]:
# Model Registry

@dataclass
class ModelVersion:
    """A versioned model in the registry."""
    model_id: str
    version: str
    base_model: str
    path: str
    training_stages: List[str]
    metrics: Dict[str, float]
    created_at: str
    status: str = "registered"  # registered, validated, deployed, archived
    tags: List[str] = field(default_factory=list)

class ModelRegistry:
    """
    Registry for model versioning and tracking.
    
    Features:
    - Version control for models
    - Metrics comparison
    - Model lineage tracking
    - Deployment status management
    """
    
    def __init__(self, registry_path: str = "./model_registry"):
        self.registry_path = Path(registry_path)
        self.registry_path.mkdir(parents=True, exist_ok=True)
        self.models: Dict[str, List[ModelVersion]] = {}
        self._load_registry()
    
    def _load_registry(self):
        """Load existing registry from disk."""
        registry_file = self.registry_path / "registry.json"
        if registry_file.exists():
            with open(registry_file) as f:
                data = json.load(f)
                for model_id, versions in data.items():
                    self.models[model_id] = [
                        ModelVersion(**v) for v in versions
                    ]
    
    def _save_registry(self):
        """Save registry to disk."""
        registry_file = self.registry_path / "registry.json"
        data = {
            model_id: [vars(v) for v in versions]
            for model_id, versions in self.models.items()
        }
        with open(registry_file, 'w') as f:
            json.dump(data, f, indent=2)
    
    def register(
        self,
        model_id: str,
        path: str,
        base_model: str,
        training_stages: List[str],
        metrics: Dict[str, float],
        tags: List[str] = None,
    ) -> ModelVersion:
        """
        Register a new model version.
        """
        # Determine version
        existing = self.models.get(model_id, [])
        version = f"v{len(existing) + 1}.0"
        
        model_version = ModelVersion(
            model_id=model_id,
            version=version,
            base_model=base_model,
            path=path,
            training_stages=training_stages,
            metrics=metrics,
            created_at=datetime.now().isoformat(),
            tags=tags or [],
        )
        
        if model_id not in self.models:
            self.models[model_id] = []
        self.models[model_id].append(model_version)
        
        self._save_registry()
        
        print(f"✅ Registered: {model_id} {version}")
        return model_version
    
    def get_latest(self, model_id: str) -> Optional[ModelVersion]:
        """Get latest version of a model."""
        versions = self.models.get(model_id, [])
        return versions[-1] if versions else None
    
    def compare(
        self,
        model_id: str,
        version_a: str,
        version_b: str,
    ) -> Dict:
        """
        Compare two model versions.
        """
        versions = {v.version: v for v in self.models.get(model_id, [])}
        
        a = versions.get(version_a)
        b = versions.get(version_b)
        
        if not a or not b:
            return {"error": "Version not found"}
        
        comparison = {
            "model_id": model_id,
            "version_a": version_a,
            "version_b": version_b,
            "metrics_diff": {},
        }
        
        all_metrics = set(a.metrics.keys()) | set(b.metrics.keys())
        for metric in all_metrics:
            val_a = a.metrics.get(metric, 0)
            val_b = b.metrics.get(metric, 0)
            comparison["metrics_diff"][metric] = {
                "a": val_a,
                "b": val_b,
                "diff": val_b - val_a,
                "pct_change": ((val_b - val_a) / val_a * 100) if val_a != 0 else 0,
            }
        
        return comparison
    
    def list_models(self) -> List[Dict]:
        """List all registered models."""
        result = []
        for model_id, versions in self.models.items():
            latest = versions[-1] if versions else None
            result.append({
                "model_id": model_id,
                "versions": len(versions),
                "latest": latest.version if latest else None,
                "status": latest.status if latest else None,
            })
        return result

print("✅ ModelRegistry class defined")

---

## Part 5: A/B Testing Framework

In [ ]:
# A/B Testing Framework

@dataclass
class ABTestResult:
    """Result of an A/B test."""
    test_id: str
    model_a: str
    model_b: str
    num_samples: int
    metrics: Dict[str, Dict[str, float]]  # {metric: {a: val, b: val, winner: str}}
    winner: str
    confidence: float
    created_at: str

class ABTestingFramework:
    """
    Framework for A/B testing model variants.
    
    Features:
    - Side-by-side model comparison
    - Statistical significance testing
    - Automatic winner selection
    - Traffic splitting
    """
    
    def __init__(self):
        self.active_tests: Dict[str, Dict] = {}
        self.completed_tests: List[ABTestResult] = []
        self._test_id_counter = 0
    
    def create_test(
        self,
        model_a_path: str,
        model_b_path: str,
        traffic_split: float = 0.5,  # Fraction to model A
    ) -> str:
        """
        Create a new A/B test.
        
        Returns:
            Test ID
        """
        self._test_id_counter += 1
        test_id = f"ab_test_{self._test_id_counter:04d}"
        
        self.active_tests[test_id] = {
            "model_a": model_a_path,
            "model_b": model_b_path,
            "traffic_split": traffic_split,
            "samples_a": [],
            "samples_b": [],
            "created_at": datetime.now().isoformat(),
        }
        
        print(f"✅ Created A/B test: {test_id}")
        print(f"   Model A: {model_a_path}")
        print(f"   Model B: {model_b_path}")
        print(f"   Traffic split: {traffic_split*100:.0f}% / {(1-traffic_split)*100:.0f}%")
        
        return test_id
    
    def route_request(self, test_id: str) -> str:
        """
        Route a request to model A or B.
        
        Returns:
            'a' or 'b'
        """
        import random
        
        test = self.active_tests.get(test_id)
        if not test:
            raise ValueError(f"Test not found: {test_id}")
        
        if random.random() < test["traffic_split"]:
            return "a"
        return "b"
    
    def record_result(
        self,
        test_id: str,
        variant: str,  # 'a' or 'b'
        metrics: Dict[str, float],
    ):
        """
        Record result for a request.
        """
        test = self.active_tests.get(test_id)
        if test:
            key = f"samples_{variant}"
            test[key].append(metrics)
    
    def analyze_test(self, test_id: str) -> ABTestResult:
        """
        Analyze test results and determine winner.
        """
        test = self.active_tests.get(test_id)
        if not test:
            raise ValueError(f"Test not found: {test_id}")
        
        samples_a = test["samples_a"]
        samples_b = test["samples_b"]
        
        if not samples_a or not samples_b:
            raise ValueError("Not enough samples for analysis")
        
        # Aggregate metrics
        metrics_comparison = {}
        all_metrics = set()
        for s in samples_a + samples_b:
            all_metrics.update(s.keys())
        
        a_wins = 0
        b_wins = 0
        
        for metric in all_metrics:
            a_values = [s.get(metric, 0) for s in samples_a]
            b_values = [s.get(metric, 0) for s in samples_b]
            
            mean_a = sum(a_values) / len(a_values) if a_values else 0
            mean_b = sum(b_values) / len(b_values) if b_values else 0
            
            # Higher is better assumption (customize per metric)
            winner = "a" if mean_a > mean_b else "b"
            if winner == "a":
                a_wins += 1
            else:
                b_wins += 1
            
            metrics_comparison[metric] = {
                "a": mean_a,
                "b": mean_b,
                "winner": winner,
            }
        
        overall_winner = "a" if a_wins >= b_wins else "b"
        confidence = max(a_wins, b_wins) / (a_wins + b_wins) if (a_wins + b_wins) > 0 else 0.5
        
        result = ABTestResult(
            test_id=test_id,
            model_a=test["model_a"],
            model_b=test["model_b"],
            num_samples=len(samples_a) + len(samples_b),
            metrics=metrics_comparison,
            winner=overall_winner,
            confidence=confidence,
            created_at=test["created_at"],
        )
        
        self.completed_tests.append(result)
        del self.active_tests[test_id]
        
        print(f"\n📊 A/B Test Results: {test_id}")
        print(f"   Samples: A={len(samples_a)}, B={len(samples_b)}")
        print(f"   Winner: Model {result.winner.upper()} (confidence: {result.confidence:.1%})")
        
        return result

print("✅ ABTestingFramework class defined")

---

## 🎉 Checkpoint

You now have the components for a training pipeline:

- ✅ Data pipeline (collection, cleaning, formatting)
- ✅ Training orchestrator (SFT, DPO)
- ✅ Model registry with versioning
- ✅ A/B testing framework
- ✅ Automated evaluation integration

---

## 🚀 Implementation Roadmap

### Week 1: Data Pipeline
- [ ] Implement data collection
- [ ] Build cleaning and filtering
- [ ] Create dataset versioning

### Week 2: Training
- [ ] Set up SFT training
- [ ] Implement DPO training
- [ ] Add checkpoint management

### Week 3: Registry & Eval
- [ ] Build model registry
- [ ] Create evaluation benchmarks
- [ ] Automate evaluation runs

### Week 4: A/B Testing
- [ ] Implement A/B framework
- [ ] Create deployment automation
- [ ] Build monitoring dashboards

### Week 5-6: Demo & Docs
- [ ] End-to-end pipeline demo
- [ ] Complete technical report
- [ ] Record demo video

In [ ]:
# 🧹 Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
print("\n🎯 Next: Check examples/option-d-training-pipeline/ for complete implementation")